MAsk Detection MOdel Using Transfer Learning

In [15]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout


In [16]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [17]:
# I have Ornize form of the data

from tensorflow.keras.preprocessing.image import ImageDataGenerator


IMG_SIZE=224
BATCH_SIZE=32
# Data Augmentation for training (rotation, zoom, etc.)

train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

# Validation & Test data ke liye sirf rescale
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)




In [18]:
train_Generator=train_datagen.flow_from_directory(
    "D:\Project-to-learn\Face_Mask_Detection_Model\Data\Train",
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=32,
    class_mode="binary"
)

from random import shuffle


val_Generator = train_datagen.flow_from_directory(
    "D:\Project-to-learn\Face_Mask_Detection_Model\Data\Validation",
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = 32,
    class_mode = "binary"
)

#

test_Generator = test_datagen.flow_from_directory(
    "D:\Project-to-learn\Face_Mask_Detection_Model\Data\Test",
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = 32,
    class_mode = "binary",
    shuffle = False
)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [19]:
# Load Pre-trained Model without top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(1, activation='sigmoid')(x)

# Final Model
model = Model(inputs=base_model.input, outputs=preds)

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [20]:
# history = model.fit(
#     train_Generator,
#     validation_data=val_Generator,
#     epochs=10
# )


In [25]:
model.save('Transfer_face_mask_detection_model.h5')

In [26]:
loss, acc = model.evaluate(test_Generator)
print(f"Saved Model Test Accuracy: {acc*100:.2f}%")


31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 204ms/step - accuracy: 0.4145 - loss: 0.8435
Saved Model Test Accuracy: 62.70%


In [27]:
# import cv2
# import numpy as np
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("Transfer_face_mask_detection_model.h5")


In [31]:
# Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start webcam
cap = cv2.VideoCapture(0)

THRESHOLD = 0.4 # try 0.6 or 0.7 if needed

while True:
    ret, frame = cap.read()
    if not ret:
        break   
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        face = face / 255.0
        face = np.expand_dims(face, axis=0)
        
        pred = model.predict(face)[0][0]
        print("Prediction:", pred)  # debug print
        
        if pred < THRESHOLD:
            label = "Mask"
            color = (0, 255, 0)
        else:
            label = "No Mask"
            color = (0, 0, 255)
        
        
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
    
    cv2.imshow("Face Mask Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: 0.5223443
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: 0.4936561
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Prediction: 0.4936561
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: 0.48505178
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Prediction: 0.53155893
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction: 0.6134312
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: 0.58069164
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: 0.3036989
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: 0.4287605
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Prediction: 0.4287605
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: 0.4507326
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Prediction: 0.46171302
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: 0.54456043
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: 0.6233786
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Prediction: 0.5168727
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: 0.58670425
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/s

KeyboardInterrupt: 